In [1]:
import argparse

from os import listdir
from os.path import join, isfile
import math
from collections import Counter

from assets.Utils import *

def getRowType(row):
    types = {   
                0: ["single"],
                1: ["double", "double"],
                2: ["double", "quadruple", "quadruple"],
                3: ["quadruple", "double", "quadruple"],
                4: ["quadruple", "quadruple", "double"],
                5: ["quadruple", "quadruple", "quadruple", "quadruple"]
            }
    
    row_children = []
    
    for child in row.children:
        row_children.append(child.tag_name)
        
    for i in range(6):
        if Counter(types[i]) == Counter(row_children):
            return i
                    
    return False
    
    
        
    
def getTokenListFromFile(filepath):
    with open(filepath) as data_file:
        tokenString = data_file.read()
    
    cleaned = tokenString.replace("},", "}")
    cleaned = cleaned.replace("{", " {")

    raw = cleaned.split("\n")

    splitted_elements = []
    for raw_element in raw:
        if len(raw_element) > 0:        
            if raw_element.find(" ") > -1:
                elements = raw_element.split(" ")
                splitted_elements.extend(elements)
            elif raw_element.find(",") > -1:
                elements = raw_element.split(",")
                splitted_elements.extend(elements)
            else:
                splitted_elements.append(raw_element)     

    processed = []
    for element in splitted_elements:
        if element != "":
            if element.find(",") > -1:
                element = element.replace(",", "")

            processed.append(element)
    
    return processed

In [8]:
predictions_with_truth = [{"truth": "./generatedMarkup/SECOND_complete_generation_4_04.10.2018_1538655331945.gui",
                           "prediction": "./generatedMarkup/second.gui"}]
path_truths = ""
path_predictions = ""

def compareResults(comparison):
    error_object = {
                        "isHeaderCorrect": None, 
                        "differenceMenuButtons" : None,
                        "differenceRowCount": None,
                        "countWrongButtons": None,
                        "countCorrectButtons": None,
                        "differenceButtonCount": None,
                        "differenceTokenCount": None,
                        "trueFileName": None,
                        "trueHeaderType": None,
                        "countWrongRowType": None,
                        "countCorrectRowType": None,
                        "predictedFileName": None
                        }

    truth = comparison["truth"]
    prediction = comparison["prediction"]

    truth_tokens = getTokenListFromFile(path_truths + truth)
    predictions_tokens = getTokenListFromFile(path_predictions + prediction)

    root_truth = Utils.createElementsFromTokenList(truth_tokens.copy())
    root_prediction = Utils.createElementsFromTokenList(predictions_tokens.copy())

    error_object["trueFileName"] = path_truths + truth
    error_object["predictedFileName"] = path_predictions + prediction

    error_object["trueHeaderType"] = root_truth.children[0].tag_name
    error_object["isHeaderCorrect"] = "sidebar" in truth_tokens and "sidebar" in predictions_tokens or "header" in truth_tokens and "header" in predictions_tokens
    error_object["differenceTokenCount"] = int( len(predictions_tokens) - len(truth_tokens))


    # COUNT MENUE BUTTONS
    if root_truth.children[0].tag_name in ["sidebar", "header"]:
        true_count_menu_buttons = len(root_truth.children[0].children)

    if root_prediction.children[0].tag_name in ["sidebar", "header"]:
        pred_count_menu_buttons = len(root_prediction.children[0].children)    

    error_object["differenceMenuButtons"] = int(pred_count_menu_buttons-true_count_menu_buttons)

    # COUNT ROWS
    true_rows = []
    for child in root_truth.children:
        if child.tag_name == "row":
            true_rows.append(child)

    pred_rows = []
    for child in root_prediction.children:
        if child.tag_name == "row":
            pred_rows.append(child)

    error_object["differenceRowCount"] = int(len(pred_rows) - len(true_rows))

    # CHECK ROW TYPES
    count_wrong_rows = 0
    count_correct_rows = 0
    for i in range(len(pred_rows)):
        if getRowType(pred_rows[i]) == getRowType(true_rows[i]):
            count_correct_rows += 1
        else:
            count_wrong_rows += 1


    error_object["countWrongRowType"] = count_wrong_rows
    error_object["countCorrectRowType"] = count_correct_rows

    # CHECK CORRECT BUTTONS
    true_buttons = []
    for row in true_rows:
        for row_element in row.children:
            for child in row_element.children:
                if "btn" in child.tag_name:
                    true_buttons.append(child.tag_name)

    pred_buttons = []
    for row in pred_rows:
        for row_element in row.children:
            for child in row_element.children:
                if "btn" in child.tag_name:
                    pred_buttons.append(child.tag_name)

    difference_button_count = int(len(pred_buttons) - len(true_buttons))

    correct_buttons = 0
    wrong_buttons = 0
    for i in range(len(pred_buttons)):
        try:
            if pred_buttons[i] == true_buttons[i]:
                correct_buttons += 1
            else:
                wrong_buttons += 1
        except Exception as ex:
            pass



    error_object["countWrongButtons"] = wrong_buttons
    error_object["countCorrectButtons"] = correct_buttons
    error_object["differenceButtonCount"] = difference_button_count

    return error_object


results = []
for comparison in predictions_with_truth:
    errors = compareResults(comparison)
    results.append(errors)



In [7]:
results

[{'countCorrectButtons': 1,
  'countCorrectRowType': 0,
  'countWrongButtons': 5,
  'countWrongRowType': 3,
  'differenceButtonCount': 3,
  'differenceMenuButtons': 0,
  'differenceRowCount': 0,
  'differenceTokenCount': 16,
  'isHeaderCorrect': True,
  'predictedFileName': './generatedMarkup/second.gui',
  'trueFileName': './generatedMarkup/SECOND_complete_generation_4_04.10.2018_1538655331945.gui',
  'trueHeaderType': 'sidebar'}]

In [10]:
comparison = {"truth": "./generatedMarkup/SECOND_complete_generation_4_04.10.2018_1538655331945.gui",
                           "prediction": "./generatedMarkup/second.gui"}

error_object = {
                    "isHeaderCorrect": None, 
                    "differenceMenuButtons" : None,
                    "differenceRowCount": None,
                    "countWrongButtons": None,
                    "countCorrectButtons": None,
                    "differenceButtonCount": None,
                    "differenceTokenCount": None,
                    "trueFileName": None,
                    "trueHeaderType": None,
                    "countWrongRowType": None,
                    "countCorrectRowType": None,
                    "predictedFileName": None,
                    "true_token_count": None
                    }

truth = comparison["truth"]
prediction = comparison["prediction"]

truth_tokens = getTokenListFromFile(path_truths + truth)
predictions_tokens = getTokenListFromFile(path_predictions + prediction)

root_truth = Utils.createElementsFromTokenList(truth_tokens.copy())
root_prediction = Utils.createElementsFromTokenList(predictions_tokens.copy())

error_object["trueFileName"] = path_truths + truth
error_object["predictedFileName"] = path_predictions + prediction

error_object["trueHeaderType"] = root_truth.children[0].tag_name
error_object["isHeaderCorrect"] = "sidebar" in truth_tokens and "sidebar" in predictions_tokens or "header" in truth_tokens and "header" in predictions_tokens
error_object["differenceTokenCount"] = int( len(predictions_tokens) - len(truth_tokens))
error_object["true_token_count"] = len(truth_tokens)


# COUNT MENUE BUTTONS
if root_truth.children[0].tag_name in ["sidebar", "header"]:
    true_count_menu_buttons = len(root_truth.children[0].children)

if root_prediction.children[0].tag_name in ["sidebar", "header"]:
    pred_count_menu_buttons = len(root_prediction.children[0].children)    

error_object["differenceMenuButtons"] = int(pred_count_menu_buttons-true_count_menu_buttons)

# COUNT ROWS
true_rows = []
for child in root_truth.children:
    if child.tag_name == "row":
        true_rows.append(child)

pred_rows = []
for child in root_prediction.children:
    if child.tag_name == "row":
        pred_rows.append(child)

error_object["differenceRowCount"] = int(len(pred_rows) - len(true_rows))

# CHECK ROW TYPES
count_wrong_rows = 0
count_correct_rows = 0
for i in range(len(pred_rows)):
    if getRowType(pred_rows[i]) == getRowType(true_rows[i]):
        count_correct_rows += 1
    else:
        count_wrong_rows += 1


error_object["countWrongRowType"] = count_wrong_rows
error_object["countCorrectRowType"] = count_correct_rows

# CHECK CORRECT BUTTONS
true_buttons = []
for row in true_rows:
    for row_element in row.children:
        for child in row_element.children:
            if "btn" in child.tag_name:
                true_buttons.append(child.tag_name)

pred_buttons = []
for row in pred_rows:
    for row_element in row.children:
        for child in row_element.children:
            if "btn" in child.tag_name:
                pred_buttons.append(child.tag_name)

difference_button_count = int(len(pred_buttons) - len(true_buttons))

correct_buttons = 0
wrong_buttons = 0
for i in range(len(pred_buttons)):
    try:
        if pred_buttons[i] == true_buttons[i]:
            correct_buttons += 1
        else:
            wrong_buttons += 1
    except Exception as ex:
        pass

error_object["countWrongButtons"] = wrong_buttons
error_object["countCorrectButtons"] = correct_buttons
error_object["differenceButtonCount"] = difference_button_count

error_object

{'countCorrectButtons': 1,
 'countCorrectRowType': 0,
 'countWrongButtons': 5,
 'countWrongRowType': 3,
 'differenceButtonCount': 3,
 'differenceMenuButtons': 0,
 'differenceRowCount': 0,
 'differenceTokenCount': 16,
 'isHeaderCorrect': True,
 'predictedFileName': './generatedMarkup/second.gui',
 'trueFileName': './generatedMarkup/SECOND_complete_generation_4_04.10.2018_1538655331945.gui',
 'trueHeaderType': 'sidebar',
 'true_token_count': 55}

In [15]:
root_prediction.toString(0)

 body 
     sidebar 
         sidebar-element WQtCc wf
         sidebar-element CLOT  1H
         sidebar-element FBz8   Xm  
         sidebar-element  nCppP7r u
     row 
         quadruple 
             small-title fv fa1Fa   hx
             text Ag0 j2t9D    oXcMi5oZu 1z IC
             btn-inactive-grey 4ToPRstJBJUk
         quadruple 
             small-title 9piQ5hSr6X I4vh
             text xPuCVwwviZh jH6aZI 4ctoY nHlj
             btn-inactive-grey g WPN b
         quadruple 
             small-title  SX5MR4qey V4a 
             text X  G  i  2 h t4qP pugA75WC7
             btn-inactive-blue  i89spsu
         quadruple 
             small-title T0qsU  PWmwhu
             text  RA L KN sn1pWb8q XQJRpTeahv
             btn-inactive-black 59 ENuS
     row 
         quadruple 
             small-title  wW 7tBbQ   s5
             text E SsHYblK  xB7 Y8Fn DCMq7Jl lQi  
             btn-inactive-grey ryAZ knj 
         quadruple 
             small-title Rxowoc ur 
             text 

In [34]:
root_truth.toString(0)

 body 
     sidebar 
         sidebar-element q7rwRy   
         sidebar-element  7 hUG7XYHY
         sidebar-element o gM r5AyBG 
         sidebar-element  i hP M
     row 
         single 
             small-title R doLWJuAqILk 
             text vMsR v 7 qYml ghTkIkU w 0 5
             btn-inactive-black  9 1a2NQmWD 
     row 
         single 
             small-title 0 F QlkBJmU3zf0
             text  2lZw z6bGPX6 w V4YkJTWP38 
             btn-inactive-blue jLoJ6o1I
     row 
         quadruple 
             small-title 1Ej6aGn4uKj t
             text tw IVVvTWkNQoxUQhXW Rpk9DmXF udG
             btn-inactive-grey 1g  E 7H
         quadruple 
             small-title uqT57YOYv v5bjk
             text  E kGd i vrMUmJ 8S7WJNyT G 6 C
             btn-inactive-grey Z  WBX0P
         quadruple 
             small-title 5oeBPn8Oe rREn
             text K swa6umoYcH W  Yz  hM lTZ  z
             btn-inactive-grey FaBVU B
         quadruple 
             small-title F VFkYL  wO Y
        